In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import math
import pylab
from scipy.interpolate import interp1d
from scipy.signal import medfilt
import time
from detector import Detector
%matplotlib notebook

In [ ]:
#need lwa software library for this
from driftcurve import driftcurve, single_time_driftcurve
import cr_data_inspection_functions as crd

In [ ]:
#driftcurve(do_plot=True, frequency=15.e6)

# part 1 : load up the data

In [ ]:
fname = '/scr/kp/cr_data/overnight_scan_20-21April2023/overnight_software_snapshots1682051758.6163533.dat'
events = crd.parsefile(fname, end_ind = 1000)
ant_id = np.array([events[i]['antenna_id'] for i in range(len(events))])
all_data_times = np.array([events[i]['timestamp'] for i in range(len(events))])
all_data_wfs = np.array([events[i]['data'] for i in range(len(events))])

In [ ]:
#select only one antenna
which_antenna = 0
cut = ant_id == which_antenna
data_times = all_data_times[cut]
data_wfs = all_data_wfs[cut]
Nevents = len(data_times)
print('number of events:', Nevents)

In [ ]:
def hanning_window(L):
    n = np.arange(L)
    return 0.5*(1-np.cos(2*np.pi*n/L))

def do_fft(fs, samples):
    '''
    returns only the positive frequencies of the FFT
    no normalization is applied
    applies a Hanning window to the FFT
    '''
    L = samples.shape[-1]

    N2 = L//2
    f_res = fs/L   # Frequency resolution

    #Y = np.fft.fft(samples*hanning_window(L), L)
    Y = np.fft.fft(samples, L)

    # Get the values of interest
    Y_amp = np.abs(Y)
    Y_phase = np.arctan2(Y.imag, Y.real)

    freq = np.arange(0, N2)*f_res

    amp = Y_amp.T[:N2].T
    ph = Y_phase.T[:N2].T

    return freq, amp, ph

In [ ]:
N = data_wfs.shape[-1]
ADC_MAX = 2**9 #10 bits, but signed
ADC_to_V = 1.35 #TODO this is a guess
fs = 196.608e6 # clock frequency from r monroe paper
all_data_freq, data_amp, data_phase = do_fft(fs, data_wfs)
#normalization to turn to Vrms
data_amp *= ADC_to_V/(ADC_MAX*N)
data_power = np.abs(data_amp)**2
#in rms V^2 at the ADC, factor of 2 is for negative freqs
all_P_data = 2. * data_power

#setting coax cable distance to 100m for now, will eventually vary with antenna
coax_distance = 100.


In [ ]:
#filter in detector.py only goes 30-80 right now
#antenna only goes 24-94
FREQ_MIN = 24.e6
FREQ_MAX = 94.e6
cut = np.logical_and(all_data_freq >= FREQ_MIN, all_data_freq <= FREQ_MAX)
data_freq = all_data_freq[cut]

#median filter spectra to remove spikes
#TODO: do real filtering
P_data = medfilt(all_P_data, [1,101])[:,cut]

#downsample frequencies to make this run faster
downsample_N = 50 #set to 1 to not downsample
data_freq =data_freq[::downsample_N]
dF = np.diff(data_freq)[0]
P_data = downsample_N * P_data[:,::downsample_N]/dF # to V^2/Hz

plt.figure()
plt.semilogy(all_data_freq/1e6, np.sqrt(all_P_data[0]), label='real trace')
plt.semilogy(data_freq/1e6, np.sqrt(P_data[0] * dF/downsample_N), label='filtered trace')
plt.xlabel("freq (MHz)")
plt.ylabel(r'Vrms at ADC input ($V_{rms}$)')
plt.legend()


# part 2 : set up the system of equations

In [ ]:
#gives skytemp in K and lst
#note that skytemp does not include antenna LNA gain
#lst, skytemp = driftcurve(do_plot=False)
det = Detector()
dfreq = data_freq[1] - data_freq[0]
P_sim = []
USE_FILTER=True
if FREQ_MIN < 30.e6 or FREQ_MAX > 80.e6:
    USE_FILTER = False # filter response in detector only runs 30-80 so we gotta fit it
for t in data_times:
    Tsky = []
    for f in data_freq:
        lst, T = single_time_driftcurve(time_start = t, frequency = f)
        Tsky.append(T)
    Tsky = np.array(Tsky)
    Psky = det.calculate_noise_power(data_freq/1e6, Tsky, dF/1e6, use_filter_datasheet=USE_FILTER)
    P_sim.append(Psky)
P_sim = np.array(P_sim).reshape(data_times.shape[0], data_freq.shape[0])

In [ ]:
#next you need to load up simulated coax cable losses to get
#currently loads up gain/100m digitized from fig 2 in the lofar calib paper

def load_coax_losses(freq, distance):
    fMHz, gdB = np.loadtxt('data/lofar_coax_per_100m.csv', delimiter=',').T
    gfunc = interp1d(fMHz, gdB, fill_value='extrapolate')
    gain_out = gfunc(freq/1e6) * distance/100.
    return 10.**(gain_out/10.)

L_coax = load_coax_losses(data_freq, coax_distance)
P_sim *= L_coax

# part 3 : use system of equations to find unknowns

In [ ]:
from scipy.optimize import curve_fit
#note that curve fit needs things flattened

def optfunc(x, *G_corr):
    # G_corr is a frequency dependent gain correction
    G = np.array(G_corr)
    y = x * G
    return y.flatten()

if Nevents < data_freq.shape[0]:
    method = 'trf'
else:
    method = 'lm'
    
popt, pcov = curve_fit(optfunc, P_sim, P_data.flatten(), p0 = 1e8*np.ones(data_freq.shape[0]))

In [ ]:
print('system gain best fit params (in dB) =\n', 10.*np.log10(popt))
if USE_FILTER:
    print('\nFilter response is NOT included in fit')
else:
    print('\nFilter response is included in fit')

In [ ]:
print('data/sim (in dB) =\n', 10.*np.log10(P_data/P_sim))